# Desafio

### Acesse a api da camara, e recupere os reembolsos do ano de 2018

In [4]:
import requests

# fazendo o download do conteúdo do arquivo
url = f"http://www.camara.leg.br/cotas/Ano-2018.csv.zip"
r = requests.get(url)


### Leia o Arquivo csv 

In [8]:
# salvando os dados em um arquivo
with open(f"Ano-2018.csv.zip", "wb") as file:

    # escrever o conteúdo
    file.write(r.content)

In [9]:
# descompactando o arquivo
from zipfile import ZipFile

zip_file = ZipFile(f"Ano-2018.csv.zip", 'r')
zip_file.extract(member=f"Ano-2018.csv", path=f"reembolso-2018")
zip_file.close()

<zipfile.ZipFile filename='Ano-2018.csv.zip' mode='r'>

In [14]:
import pandas as pd

data = pd.read_csv("reembolso-2018/Ano-2018.csv", delimiter=';')

In [29]:
data.shape

(292655, 31)

### Visualize 10 linhas aleatórias

In [16]:
data.sample(10)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
180166,Roberto de Lucena,5.781728e+09,160653.0,385.0,2015,SP,PODE,55,11,SERVIÇOS POSTAIS,...,2018,0,NaN,NaN,0,0.0,NaN,2282,0,NaN
222038,Helder Salomão,7.680874e+10,178873.0,279.0,2015,ES,PT,55,122,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",...,2018,0,NaN,NaN,1537370,NaN,NaN,2894,6694137,https://www.camara.leg.br/cota-parlamentar/doc...
159739,Zeca Dirceu,3.098872e+09,160592.0,468.0,2015,PR,PT,55,14,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO ...",...,2018,0,NaN,NaN,1540500,NaN,NaN,2238,6702472,https://www.camara.leg.br/cota-parlamentar/doc...
44615,Julio Lopes,5.924522e+10,74253.0,586.0,2015,RJ,PP,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,...,2018,0,NaN,NaN,1466733,NaN,NaN,1576,6507503,https://www.camara.leg.br/cota-parlamentar/doc...
254977,Marcos Soares,5.217857e+09,178943.0,574.0,2015,RJ,DEM,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,2018,0,NaN,NaN,1496930,NaN,NaN,2911,6586025,https://www.camara.leg.br/cota-parlamentar/doc...
271692,Eduardo Cury,4.909671e+09,178977.0,353.0,2015,SP,PSDB,55,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2018,0,NaN,NaN,1544004,NaN,NaN,2982,6711871,https://www.camara.leg.br/cota-parlamentar/doc...
234226,ZECA DO PT,4.064992e+09,178902.0,440.0,2015,MS,PT,55,14,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO ...",...,2018,0,NaN,NaN,1464457,NaN,NaN,2944,6501388,https://www.camara.leg.br/cota-parlamentar/doc...
278210,Nilto Tatto,3.380917e+09,178986.0,382.0,2015,SP,PT,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,...,2018,0,NaN,NaN,1473195,NaN,NaN,3072,6523897,http://camara.leg.br/cota-parlamentar/nota-fis...
222243,Sergio Vidigal,5.254981e+10,178874.0,283.0,2015,ES,PDT,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,2018,0,NaN,NaN,1460377,NaN,NaN,3046,6490271,https://www.camara.leg.br/cota-parlamentar/doc...
220524,Evair Vieira de Melo,2.261266e+09,178871.0,274.0,2015,ES,PP,55,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2018,0,NaN,NaN,1504549,NaN,NaN,2984,6603722,https://www.camara.leg.br/cota-parlamentar/doc...


### Existem registros com `ideDocumento` nulo?

In [20]:
data['ideDocumento'].isnull().sum()

0

### Converta a coluna `datEmissao` para `datetime`, e descubra quantos reembolsos aconteram no primeiro semestre

In [23]:
data['datEmissao'] = pd.to_datetime(
    data.datEmissao, 
    format='%Y-%m-%d'
) 

In [28]:
data[data['datEmissao']<'2018-07-01'].shape

(159782, 31)

### Limpe a coluna `txtCNPJCPF`: ela deve conter somente dígitos numéricos

In [34]:
# vamos trocar qualquer caracter que não seja um dígito número por '', essa operação remove os dígitos
data["txtCNPJCPF"] = data["txtCNPJCPF"].str.replace(r'\D','',regex=True)

In [35]:
data["txtCNPJCPF"]

0         04644820000177
1         10638597000158
2         24882567000105
3         24882567000105
4         03713903000108
               ...      
292650    00000000000006
292651    00982933000121
292652    27402097000115
292653    02214932000162
292654    02012862000160
Name: txtCNPJCPF, Length: 292655, dtype: object

### Quais são os valores únicos do campo `indTipoDocumento`? Substitua o conteúdo pela respectiva legenda

*Dica*: 0 (Zero), para Nota Fiscal; 1 (um), para Recibo; e 2, para Despesa no Exterior.

In [37]:
data['indTipoDocumento'].unique()

array([4, 0, 1, 2, 3])

In [38]:
legenda = {0:'Nota Fiscal',1:'Recibo',2:'Despesas no Exterior'}

In [40]:
data['indTipoDocumento'].replace(legenda,inplace=True)

In [41]:
data['indTipoDocumento'].unique()

array([4, 'Nota Fiscal', 'Recibo', 'Despesas no Exterior', 3],
      dtype=object)